In [ ]:
from IPython.display import Image
from IPython.display import display_html
from IPython.display import display
from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML

<h1> What is Matrix Completion? </h1>
<p> Simply put, the goal of matrix completion is fill in missing entries of a matrix (or dataset) given the fact that the matrix is low rank, or low dimensional.  Essentially, it's like a game of Sudoku with a different set of rules. Lets say I have a matrix that I know is supposed to be rank 2.  That means that every column can be written as a linear combination (weighted sum) of two vectors.  Lets look at an example of what this puzzle might look like.  </p>

$$ \begin{bmatrix}   
1 & 1 &2 & 2\\
2&1&3&\\
1&2&&1\\
\end{bmatrix}$$

<p> The first two columns are completly filled in, so we can use those to figure out the rest of the columns.  Based on the few entries in the third column that are given, we can see that the third column should probably be the first column plus the second column.  Likewise, the fourth column is two times the first column minus the second column. </p>
    
$$ \begin{bmatrix}   
1 & 1 &2 & 2\\
2&1&3&5\\
1&2&3&1\\
\end{bmatrix}$$

<p> That was a particularly easy example since we knew the first two columns completely. </p>  

    
<p> To see why we should care about this, here's a claim that shouldn't be too hard to believe: <b> Datasets are inherently low rank </b>.  In the example we just did, the columns could be movies, the rows could be people, and the numbers could be how each person rated each movie.  Obviously, this is going to be sparse since not everyone has seen every movie.  That's where matrix completions comes in.  When we filled in the missing entries, we gave our guess as to what movies people are going to enjoy. After explaining an algorithm to do matrix completion, we're going to try this for a data set with a million ratings people gave movies and see how well we recommend movies to people.</p>
  
<h1> How do we do it? </h1>

There's two paradigms for matrix completion.  One is to minimize the rank of a matrix that fits our measurements, and the other is to find a matrix of a given rank that matches up with our known entries.  In this blog post, I'll just be talking about the second.  

Before we explain the algorithm, we need to introduce a little more notation. We are going to let $\Omega$ be the set of indices where we know the entry.  For example, if we have the partially observed matrix
$$ \begin{matrix}
\color{blue}1\\\color{blue}2\\\color{blue}3
\end{matrix}
\begin{bmatrix}   
  & 1 &  \\
  &   & 1\\
1 &   &  
  \end{bmatrix}$$
    
  $$ 
\begin{matrix}   
 &\color{red}1 & \color{red}2 & \color{red}3  \end{matrix}$$
then, $\Omega$ would be $\{ (\color{blue} 1, \color{red}2), (\color{blue}2 , \color{red}3),(\color{blue} 3, \color{red}1)\}$  We can now pose the problem of finding a matrix with rank $r$ that best fits the entries we've observe as an <i> optimization problem</i>.  
$$
\begin{align}
&\underset{X}{\text{minimize}}& \sum_{(i,j)\text{ in }\Omega} (X_{ij}-M_{ij})^2 \\
& \text{such that} & \text{rank}(X)=r
\end{align}
$$
The first line specifies <i> objective function </i>(the function we want to minimize), which is the sum of the square of the difference between $X_{ij}$ and $M_{ij}$ for every $(i,j)$ that we have a measurement for.  The second line is our <i> constraint</i>, which says that the matrix has to be rank $r$.

While minimizing a function like that isn't too hard, forcing the matrix to be rank $r$ can be tricky. One property of a low rank matrix that has $m$ rows and $n$ columns is that we can factor it into two smaller matrices like such: 
$$X=UV$$
where $U$ is $n$ by $r$ and $V$ is $r$ by $m$.  So now, if we can find matrices $U$ and $V$ such that the matrix $UV$ fits our data, we know its going to be rank $r$ and that will be the solution to our problem. 

If $u_i$ is the $i^{th}$ column of $U$ and $v_j$ is the $j^{th}$ column of $V$, then $X_{ij}$ is the <i> inner product </i> of $u_i$ and $v_j$, $X_{ij}= \langle u_i, v_i \rangle$.  We can rewrite the optimization problem we want to solve as 
$$
\begin{align}
&\underset{U, V}{\text{minimize}}& \sum_{(i,j)\in \Omega} (\langle u_i, v_i \rangle-M_{ij})^2 
\end{align}
$$
In order to solve this, we can alternate between optimizing for $U$ while letting $V$ be a constant, and optimizing over $V$ while letting $U$ be a constant.  If $t$ is the iteration number, then the algorithm is simply 
$$
\begin{align}
\text{for } t=1,2,\ldots:& \\
U^{t}=&\underset{U}{\text{minimize}}& \sum_{(i,j)\in \Omega} (\langle u_i, v^{t-1}_i \rangle-M_{ij})^2 \\
V^{t}=&\underset{ V}{\text{minimize}}& \sum_{(i,j)\in \Omega} (\langle u^t_i, v_i \rangle-M_{ij})^2 \\
\end{align}
$$
At each iteration, we just need to solve a least squares problem which is easy enough.  




In [ ]:
import numpy as np
from scipy.optimize import minimize

def alt_min(m,n,r, Omega, known):
    U=np.random.rand(m,r)
    V=np.random.rand(r,n)

    for i in range(0,100):   
        
        objU=lambda x: np.linalg.norm(np.reshape(x, [m,r]).dot(V)[Omega]-known)**2
        U = np.reshape(minimize(objU, U).x, [m,r])
        
        objV=lambda x: np.linalg.norm(U.dot(np.reshape(x, [r,n]))[Omega]-known)**2
        V = np.reshape(minimize(objV, V).x, [r,n])

        res=np.linalg.norm(U.dot(V)[Omega]-known)
        if res < 0.0001:
            break
    return (U,V)

Lets test our algorithm with the simple example given earlier.

In [ ]:
X=([0,0,0,0,1,1,1,2,2,2], [0,1,2,3,0,1,2,0,1,3])
y=[1,1,2,2,2,1,3,1,2,1]
(U,V)=alt_min(3,4,2,X, y)
print(U.dot(V))

Thats the same matrix we came up with!



No parameters are required when you initialize, but a few you might want to specify are
 - <i>method</i>: there are a few different methods you could use. 
  - <i>AltMin</i>: Essentially what I talked about in this post
  - <i>NuclearNorm</i>: This technique minimizes the rank instead of fitting a matrix to a fixed rank.  It uses a relaxation of the rank function called the nuclear norm, But i would advise instead using NonconvexeReg
  - <i>NonconvexReg</i>: this uses a better relaxation of the rank function, which my PhD research is focused on. 
 - <i> eps </i>: the average amount of error you would expect in any individual entry.  The algorithm will pick a rank as small as possible so that the average error is no larger than <i>eps</i>.  If you make it too small, you're more likely to overfit your data, and if you make it too large, the 
 - <i>r</i>: an upper bound on the rank of the matrix.  Making this smaller will make the algorithm run faster, but making it too small might oversimplify the model and give bad results.
 
